# 라이브러리

In [62]:
import pandas as pd
import os
import tqdm
import numpy as np
from datetime import datetime

In [63]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
raw_fold = 'D:/data/big2/BD1/raw'

# Load

## result(전체)

In [72]:
# about 16.9s
name = '4등급_차량_리스트_양식_result_최종(전체)(2023.04.18_PM.02.12.53)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 36 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   차대번호        1502235 non-null  object 
 1   차량번호        1502235 non-null  object 
 2   제원관리번호      1502235 non-null  object 
 3   차종          1502235 non-null  object 
 4   용도          1502235 non-null  object 
 5   최초등록일       1502235 non-null  int64  
 6   연식          1502235 non-null  int64  
 7   제작일자        1335178 non-null  float64
 8   검사유효일       1502223 non-null  float64
 9   배출가스인증번호    1384783 non-null  object 
 10  등급          1502235 non-null  int64  
 11  차명          1502235 non-null  object 
 12  차종분류        1502235 non-null  object 
 13  차종유형        1502235 non-null  object 
 14  자동차형식       1501958 non-null  object 
 15  제작사명        1492590 non-null  object 
 16  연료          1502235 non-null  object 
 17  엔진형식        1500400 non-null  object 
 18  총중량         1502234 no

## 법정동코드 표

In [74]:
name = 'tb_mapdata_gu(2023.04.20)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(raw_fold, file_name))

In [75]:
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dong_code    278 non-null    int64  
 1   ctprvn_code  278 non-null    int64  
 2   signgu_code  278 non-null    int64  
 3   sdstrd_code  278 non-null    int64  
 4   ctprvn_nm    278 non-null    object 
 5   signgu_nm    260 non-null    object 
 6   sdstrd_nm    0 non-null      float64
dtypes: float64(1), int64(4), object(2)
memory usage: 15.3+ KB


In [76]:
code.shape

(278, 7)

In [77]:
code.head()

,dong_code,ctprvn_code,signgu_code,sdstrd_code,ctprvn_nm,signgu_nm,sdstrd_nm
0,0,0,0,0,전국,NaN,NaN
1,1100000000,11,0,0,서울특별시,NaN,NaN
2,1111000000,11,110,0,서울특별시,종로구,NaN
3,1114000000,11,140,0,서울특별시,중구,NaN
4,1117000000,11,170,0,서울특별시,용산구,NaN


# 확인

In [78]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고'],
      dtype='object')

In [79]:
code.columns

Index(['dong_code', 'ctprvn_code', 'signgu_code', 'sdstrd_code', 'ctprvn_nm',
       'signgu_nm', 'sdstrd_nm'],
      dtype='object')

In [80]:
df[['시도명', '시군구명', '법정동코드']].isnull().sum()

시도명      0
시군구명     0
법정동코드    0
dtype: int64

In [81]:
code[['ctprvn_nm', 'signgu_nm', 'dong_code']].isnull().sum()

ctprvn_nm     0
signgu_nm    18
dong_code     0
dtype: int64

# 전처리

In [82]:
code.head()

,dong_code,ctprvn_code,signgu_code,sdstrd_code,ctprvn_nm,signgu_nm,sdstrd_nm
0,0,0,0,0,전국,NaN,NaN
1,1100000000,11,0,0,서울특별시,NaN,NaN
2,1111000000,11,110,0,서울특별시,종로구,NaN
3,1114000000,11,140,0,서울특별시,중구,NaN
4,1117000000,11,170,0,서울특별시,용산구,NaN


In [83]:
code = code.rename(columns={'ctprvn_nm':'시도명', 'signgu_nm':'시군구명'})
code.head()

,dong_code,ctprvn_code,signgu_code,sdstrd_code,시도명,시군구명,sdstrd_nm
0,0,0,0,0,전국,NaN,NaN
1,1100000000,11,0,0,서울특별시,NaN,NaN
2,1111000000,11,110,0,서울특별시,종로구,NaN
3,1114000000,11,140,0,서울특별시,중구,NaN
4,1117000000,11,170,0,서울특별시,용산구,NaN


In [84]:
df[['법정동코드', '시도명','시군구명']].head()

,법정동코드,시도명,시군구명
0,3017011600,대전광역시,서구
1,2635010700,부산광역시,해운대구
2,4148012200,경기도,파주시
3,2623010900,부산광역시,부산진구
4,5013010300,제주특별자치도,서귀포시


## 시도명 시군구명 수정

### 확인

In [85]:
df['시도명'].unique()

array(['대전광역시', '부산광역시', '경기도', '제주특별자치도', '경상남도', '강원도', '전라남도', '인천광역시',
       '서울특별시', '충청북도', '경상북도', '충청남도', '대구광역시', '전라북도', '울산광역시', '광주광역시',
       '세종특별자치시'], dtype=object)

In [86]:
code['시도명'].unique()

array(['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시',
       '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도',
       '경상북도', '경상남도', '제주특별자치도'], dtype=object)

In [87]:
len(set(df['시도명'].unique()) - set(code['시도명'].unique()))

0

In [88]:
df['시군구명'].unique()

array(['서구', '해운대구', '파주시', '부산진구', '서귀포시', '이천시', '진주시', '여주시', '원주시',
       '무안군', '남양주시', '화성시', '안성시', '평택시', '안산시 단원구', '강화군', '서초구', '금정구',
       '성남시 분당구', '제주시', '정선군', '강남구', '금천구', '순천시', '창원시 마산회원구', '제천시',
       '용인시 수지구', '봉화군', '광주시', '울진군', '포항시 북구', '대덕구', '군포시', '용인시 기흥구',
       '고성군', '화순군', '구례군', '서대문구', '아산시', '영동군', '미추홀구', '충주시', '마포구',
       '홍성군', '양산시', '성남시 수정구', '은평구', '부천시', '달성군', '안양시 동안구', '전주시 덕진구',
       '하남시', '수성구', '북구', '송파구', '유성구', '달서구', '남동구', '오산시', '김해시',
       '춘천시', '창녕군', '고양시 일산서구', '강서구', '진천군', '부평구', '강릉시', '익산시', '문경시',
       '청주시 서원구', '칠곡군', '중구', '동구', '광명시', '영등포구', '상주시', '가평군',
       '성남시 중원구', '홍천군', '도봉구', '수원시 영통구', '거제시', '양천구', '시흥시', '종로구',
       '수영구', '남원시', '성동구', '남구', '창원시 마산합포구', '양주시', '세종특별자치시',
       '용인시 처인구', '동대문구', '예산군', '경주시', '동해시', '중랑구', '노원구', '포항시 남구',
       '창원시 성산구', '거창군', '전주시 완산구', '광진구', '김천시', '해남군', '수원시 팔달구',
       '고양시 일산동구', '청주시 상당구', '정읍시', '동작구', '연천군', '광산구', '완도군', '논산시',
  

In [89]:
code['시군구명'].unique()

array([nan, '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '서구', '동구',
       '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '연제구',
       '수영구', '사상구', '기장군', '수성구', '달서구', '달성군', '미추홀구', '연수구', '남동구',
       '부평구', '계양구', '강화군', '옹진군', '광산구', '유성구', '대덕구', '울주군', '수원시',
       '수원시 장안구', '수원시 권선구', '수원시 팔달구', '수원시 영통구', '성남시', '성남시 수정구',
       '성남시 중원구', '성남시 분당구', '의정부시', '안양시', '안양시 만안구', '안양시 동안구', '부천시',
       '광명시', '평택시', '동두천시', '안산시', '안산시 상록구', '안산시 단원구', '고양시',
       '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시', '구리시', '남양주시', '오산시',
       '시흥시', '군포시', '의왕시', '하남시', '용인시', '용인시 처인구', '용인시 기흥구', '용인시 수지구',
       '파주시', '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시',
       '연천군', '가평군', '양평군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시',
       '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군',
       '인제군

In [90]:
# 데이터에는 있지만 코드집에는 없는 지역
# 부천시 소사구 -> 부천시
# 부천시 오정구 -> 부천시
# 부천시 원미구 -> 부천시
# 세종특별자치시 -> nan
# 여주군 -> 여주시
# 청원군 -> 청주시
set(df['시군구명'].unique()) - set(code['시군구명'].unique())

{'부천시 소사구', '부천시 오정구', '부천시 원미구', '세종특별자치시', '여주군', '청원군'}

In [91]:
df.loc[df['시군구명'] == '세종특별자치시', '시도명'].unique()

array(['세종특별자치시'], dtype=object)

In [92]:
code.loc[code['시도명'] == '세종특별자치시', '시군구명'].unique()

array([nan], dtype=object)

In [93]:
df.loc[df['시군구명'] == '여주군', '시도명'].unique()

array(['경기도'], dtype=object)

In [94]:
df.loc[df['시군구명'] == '청원군', '시도명'].unique()

array(['충청북도'], dtype=object)

In [95]:
df.loc[df['시군구명'] == '청주시', '시도명'].unique()

array([], dtype=object)

### 수정
- 부천시 소사구 -> 부천시
- 부천시 오정구 -> 부천시
- 부천시 원미구 -> 부천시
- 세종특별자치시 -> nan
- 여주군 -> 여주시
- 청원군 -> 청주시
- 인천광역시 남구 -> 인천광역시 미추홀구

In [125]:
df['시군구명_수정'] = df['시군구명']
df['시군구명_수정'].isnull().sum()

0

In [126]:
df.loc[df['시군구명'] == '부천시 소사구', '시군구명'].shape

(6,)

In [127]:
df.loc[df['시군구명'] == '부천시 오정구', '시군구명'].shape

(19,)

In [128]:
df.loc[df['시군구명'] == '부천시 원미구', '시군구명'].shape

(20,)

In [129]:
df.loc[df['시군구명'] == '부천시 소사구', '시군구명_수정'] = '부천시'
df.loc[df['시군구명'] == '부천시 오정구', '시군구명_수정'] = '부천시'
df.loc[df['시군구명'] == '부천시 원미구', '시군구명_수정'] = '부천시'

In [130]:
df.loc[df['시군구명'] == '세종특별자치시', '시군구명'].shape

(8122,)

In [131]:
df.loc[df['시군구명'] == '세종특별자치시', '시군구명_수정'] = np.nan

In [132]:
df.loc[df['시군구명'] == '여주군', '시군구명'].shape

(1,)

In [133]:
df.loc[df['시군구명'] == '여주군', '시군구명_수정'] = '여주시'

In [134]:
df.loc[df['시군구명'] == '청원군', '시군구명'].shape

(4,)

In [135]:
df.loc[df['시군구명'] == '청원군', '시군구명_수정'] = '청주시'

In [145]:
df.loc[(df['시도명'] == '인천광역시') & (df['시군구명'] == '남구'), '시군구명'].shape

(9,)

In [146]:
df.loc[(df['시도명'] == '인천광역시') & (df['시군구명'] == '남구'), '시군구명_수정'] = '미추홀구'

In [147]:
set(df['시군구명_수정'].unique()) - set(code['시군구명'].unique())

set()

In [148]:
df['시군구명_수정'].isnull().sum()

8122

### 병합

In [149]:
df.shape

(1502235, 37)

In [150]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', '시군구명_수정'],
      dtype='object')

In [151]:
code.columns

Index(['dong_code', 'ctprvn_code', 'signgu_code', 'sdstrd_code', '시도명', '시군구명',
       'sdstrd_nm'],
      dtype='object')

In [152]:
total = df.merge(code[['시도명', '시군구명', 'dong_code']], left_on=['시도명', '시군구명_수정'], right_on=['시도명', '시군구명'], how='left')
total.shape

(1502235, 39)

In [153]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명_x', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', '시군구명_수정', '시군구명_y', 'dong_code'],
      dtype='object')

In [154]:
total['dong_code'].isnull().sum()

0

In [156]:
total = total.rename(columns={'시군구명_x':'시군구명'})
total = total.drop(['시군구명_수정', '시군구명_y'], axis=1)
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'dong_code'],
      dtype='object')

In [157]:
total[['법정동코드', '시도명','시군구명', 'dong_code']].head()

,법정동코드,시도명,시군구명,dong_code
0,3017011600,대전광역시,서구,3017000000
1,2635010700,부산광역시,해운대구,2635000000
2,4148012200,경기도,파주시,4148000000
3,2623010900,부산광역시,부산진구,2623000000
4,5013010300,제주특별자치도,서귀포시,5013000000


In [158]:
code.loc[(code['시도명'] == '대전광역시') & (code['시군구명'] == '서구'), 'dong_code']

73    3017000000
Name: dong_code, dtype: int64

In [159]:
code.loc[(code['시도명'] == '부산광역시') & (code['시군구명'] == '해운대구'), 'dong_code']

36    2635000000
Name: dong_code, dtype: int64

In [161]:
total.loc[(total['시도명'] == '인천광역시') & (total['시군구명'] == '남구'), ['시도명', '시군구명', 'dong_code']]

,시도명,시군구명,dong_code
1216692,인천광역시,남구,2817700000
1224632,인천광역시,남구,2817700000
1231283,인천광역시,남구,2817700000
1238582,인천광역시,남구,2817700000
1255630,인천광역시,남구,2817700000
1428021,인천광역시,남구,2817700000
1445600,인천광역시,남구,2817700000
1454795,인천광역시,남구,2817700000
1454818,인천광역시,남구,2817700000


In [162]:
code.loc[(code['시도명'] == '인천광역시') & (code['시군구명'] == '미추홀구'), ['시도명', '시군구명', 'dong_code']]

,시도명,시군구명,dong_code
56,인천광역시,미추홀구,2817700000


In [ ]:
total.loc[total['시도명'] == '세종특별자치시', ['시도명', '시군구명', 'dong_code']].head()

In [ ]:
code.loc[code['시도명'] == '세종특별자치시', ['시도명', '시군구명', 'dong_code']]

## 출력

In [164]:
total.shape

(1502235, 37)

In [165]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'dong_code'],
      dtype='object')

In [163]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.04.21_AM.09.53.18'

In [166]:
# about 22.4s
name = f'4등급_차량_리스트_양식_result_최종(전체)(dong_code추가)({today_date})'
file_name = f'{name}.csv'
total.to_csv(os.path.join(raw_fold, file_name), index=False, encoding='cp949')

In [167]:
name

'4등급_차량_리스트_양식_result_최종(전체)(dong_code추가)(2023.04.21_AM.09.53.18)'

In [168]:
# about 14m 16.4s
per = 700000 # 70만
total.iloc[:per].to_excel(os.path.join(raw_fold, f'{name}01.xlsx'), index=False)
total.iloc[per:].to_excel(os.path.join(raw_fold, f'{name}02.xlsx'), index=False)

# 코드 마지막